# Prédiction d'un résultat

In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn import metrics

### Importation du csv. Par défaut, le fichier que l'on récupère provient de github, raw indique que ce ne sont que les données brutes, sans les balises html.

__Si jamais vous ne possédez pas internet, commentez le premier read_csv, et décommentez le second__

In [61]:
data_result_match = pd.read_csv("https://raw.githubusercontent.com/Phignis/predict-women-soccer-results/main/src/ressources/data/woman_international_cup_results.csv", sep=",")
# data_result_match = pd.read_csv("ressources/data/woman_international_cup_results.csv", sep=",")

In [62]:
print(data_result_match.shape)

(4296, 9)


In [63]:
# print(data_result_match)
data_result_match.head(10)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1969-11-01,Italy,France,1,0,Euro,Novara,Italy,False
1,1969-11-01,Denmark,England,4,3,Euro,Aosta,Italy,True
2,1969-11-02,England,France,2,0,Euro,Turin,Italy,True
3,1969-11-02,Italy,Denmark,3,1,Euro,Turin,Italy,False
4,1975-08-25,Thailand,Australia,3,2,AFC Championship,Hong Kong,Hong Kong,True
5,1975-08-25,Hong Kong,New Zealand,0,2,AFC Championship,Hong Kong,Hong Kong,False
6,1975-08-27,Thailand,Singapore,3,0,AFC Championship,Hong Kong,Hong Kong,True
7,1975-08-27,New Zealand,Malaysia,3,0,AFC Championship,Hong Kong,Hong Kong,True
8,1975-08-29,Australia,Singapore,3,0,AFC Championship,Hong Kong,Hong Kong,True
9,1975-08-29,Hong Kong,Malaysia,0,2,AFC Championship,Hong Kong,Hong Kong,False


In [64]:

""" On ajoute une colonne Win pour savoir quelle équipe a gagné. 
 0 égalité
 1 victoire home
 2 victoire away """
print(data_result_match.shape[0])

for index in data_result_match.index :
    if(data_result_match.loc[index, 'home_score'] == data_result_match.loc[index, 'away_score']) :
      win[index] = 0
    elif(data_result_match.loc[index, 'home_score'] > data_result_match.loc[index, 'away_score']) :
      win[index] = 1
    else :
      win[index] = 2

data_result_match['Win'] = win


4296


In [65]:
array_temp = np.array(data_result_match[['away_team', 'home_team']])

print(len(pd.unique(data_result_match['home_team'])))
print(len(pd.unique(data_result_match['away_team'])))

print(set(np.concatenate([pd.unique(data_result_match['home_team']), pd.unique(data_result_match['away_team'])])))

188
184
{'British Virgin Islands', 'Croatia', 'Costa Rica', 'Netherlands Antilles', 'Singapore', 'U.S. Virgin Islands', 'Faroe Islands', 'Czechoslovakia', 'São Tomé and Príncipe', 'Saint Vincent and the Grenadines', 'Mexico', 'Benin', 'Slovenia', 'Uganda', 'Guam', 'Turkey', 'Macedonia', 'Sweden', 'Papua New Guinea', 'Tajikistan', 'Lesotho', 'Great Britain', 'Serbia', 'Slovakia', 'Suriname', 'Vanuatu', 'Iran', 'Azerbaijan', 'Guinea', 'Cameroon', 'Bosnia and Herzegovina', 'Japan', 'Djibouti', 'Andorra', 'Jordan', 'El Salvador', 'Namibia', 'Kuwait', 'Wales', 'Honduras', 'Guyana', 'Guadeloupe', 'Grenada', 'Kosovo', 'Luxembourg', 'Trinidad and Tobago', 'China PR', 'Tonga', 'Belize', 'Curaçao', 'Thailand', 'Indonesia', 'Panama', 'Bolivia', 'Ethiopia', 'Kyrgyzstan', 'Congo', 'Mauritius', 'New Zealand', 'South Korea', 'Belarus', 'Senegal', 'Malawi', 'North Korea', 'Nigeria', 'Mozambique', 'Myanmar', 'Nepal', 'India', 'Tunisia', 'South Africa', 'Ecuador', 'Bulgaria', 'Colombia', 'Bahamas', 'Phi

In [66]:

""" On ajoute une colonne id_away_team et id_home_team pour connaitre les équipes ayant joué, sous forme numérique pour traitement"""

id_home_team = []
id_away_team = []
for index in data_result_match.index :
 id_home_team += [hash(data_result_match.loc[index, 'home_team'])]
 id_away_team += [hash(data_result_match.loc[index, 'away_team'])]

data_result_match['id_home_team'] = id_home_team
data_result_match['id_away_team'] = id_away_team

data_result_match.head(10)


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,Win,id_home_team,id_away_team
0,1969-11-01,Italy,France,1,0,Euro,Novara,Italy,False,1,-3779540873683691148,-3639685518696651049
1,1969-11-01,Denmark,England,4,3,Euro,Aosta,Italy,True,1,8762829069385907453,6659840675732466
2,1969-11-02,England,France,2,0,Euro,Turin,Italy,True,1,6659840675732466,-3639685518696651049
3,1969-11-02,Italy,Denmark,3,1,Euro,Turin,Italy,False,1,-3779540873683691148,8762829069385907453
4,1975-08-25,Thailand,Australia,3,2,AFC Championship,Hong Kong,Hong Kong,True,1,-650581084020804468,5356949618074078544
5,1975-08-25,Hong Kong,New Zealand,0,2,AFC Championship,Hong Kong,Hong Kong,False,2,2067571297505163692,7503295287825894557
6,1975-08-27,Thailand,Singapore,3,0,AFC Championship,Hong Kong,Hong Kong,True,1,-650581084020804468,-62408315286070254
7,1975-08-27,New Zealand,Malaysia,3,0,AFC Championship,Hong Kong,Hong Kong,True,1,7503295287825894557,-1693305332219656756
8,1975-08-29,Australia,Singapore,3,0,AFC Championship,Hong Kong,Hong Kong,True,1,5356949618074078544,-62408315286070254
9,1975-08-29,Hong Kong,Malaysia,0,2,AFC Championship,Hong Kong,Hong Kong,False,2,2067571297505163692,-1693305332219656756


In [67]:
#data_result_match['id_home_team'] = hash(data_result_match['home_team'])
#data_result_match['id_away_team'] = hash(data_result_match['away_team'])

#

In [68]:
""" Ici nous choisissons d'utiliser 75% du datatset pour entraîner le modèle et 25% pour effectuer nos tests."""
# x_train, x_test, y_train, y_test = train_test_split(data_result_match.loc[:, data_result_match.columns.difference(['date', 'Win', 'home_team', 'away_team', 'city', 'country', 'tournament', 'away_score', 'home_score'])], data_result_match['Win'], test_size = 0.25, random_state = 0)
x_train, x_test, y_train, y_test = train_test_split(data_result_match[['neutral', 'id_home_team', 'id_away_team']], data_result_match['Win'], test_size = 0.25, random_state = 0)



In [69]:
""" Nous entrainons le modèle en utilisant la méthode de classification avecc l'arbre de décision"""

arbre_decision = DecisionTreeClassifier(random_state = 0, max_depth = 20)
clf = arbre_decision.fit(x_train, y_train)




In [70]:
""" Nous effectuons notre prédiction sur notre jeu de test en utilisant le modèle entrainé avec la méthode de classification avecc l'arbre de décision """
sPredict = clf.predict(x_test)

score = accuracy_score(y_test, sPredict)

print(score)
print(metrics.confusion_matrix(y_test, sPredict))

print(y_test.head(10))
# for x in sPredict :
  # print(x)

0.542830540037244
[[ 26  87  37]
 [ 71 402 101]
 [ 71 124 155]]
545     2
2232    2
3222    1
2194    1
2971    1
1746    2
3438    0
1811    1
220     1
3932    2
Name: Win, dtype: int64


In [71]:
""" Nous entrainons le modèle en utilisant la méthode de classification avec le plus proche voision """

KNN = KNeighborsClassifier()
clf = KNN.fit(x_train, y_train)


In [72]:
""" Nous effectuons notre prédiction sur notre jeu de test en utilisant le modèle entrainé avec la méthode de classification avec le plus proche de voisin """

sPredict = clf.predict(x_test)

score = accuracy_score(y_test, sPredict);
print(score)

print(metrics.confusion_matrix(y_test, sPredict))


0.5279329608938548
[[ 12 103  35]
 [ 36 420 118]
 [ 34 181 135]]


In [73]:

""" Nous entrainons le modèle en utilisant la méthode de classification avec SVM """
clf = svm.SVC(gamma = 0.001)
clf.fit(x_train, y_train)


SVC(gamma=0.001)

In [74]:
""" Nous effectuons notre prédiction sur notre jeu de test en utilisant le modèle entrainé avec la méthode de classification avec SVM """

sPredict = clf.predict(x_test)

score = accuracy_score(y_test, sPredict);
print(score)
print(type(x_test))

0.5921787709497207
<class 'pandas.core.frame.DataFrame'>


SyntaxError: ignored